# Digging user information

Chatbot harus:
1. Mengajukan pertanyaan tentang pertanyaan tentang aktivitas harian, kondisi emosi, preferensi, kondisi fisik, dan lingkungan pengguna. (kualitatif atau kuantitatif)
2. Menyimpan riwayat percakapan (disimpan di database atau untuk respon selanjutnya)

In [1]:
import openai
import sqlite3
from datetime import datetime

openai.api_key = "YOUR_API_KEY"

# Inisialisasi database
def init_db():
    conn = sqlite3.connect("chatbot.db")
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS conversation_history (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_input TEXT,
        chatbot_response TEXT,
        timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
    )
    """)
    conn.commit()
    conn.close()

# Fungsi untuk menyimpan riwayat percakapan
def save_to_db(user_input, chatbot_response):
    conn = sqlite3.connect("chatbot.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO conversation_history (user_input, chatbot_response) VALUES (?, ?)",
                   (user_input, chatbot_response))
    conn.commit()
    conn.close()

# Daftar pertanyaan yang akan diajukan
questions = [
    "Bagaimana aktivitas Anda hari ini?",
    "Bagaimana perasaan Anda saat ini?",
    "Apakah ada masalah atau kekhawatiran yang Anda rasakan?",
    "Seberapa puas Anda dengan aktivitas harian Anda dari skala 1-10?",
    "Bagaimana kondisi fisik Anda hari ini? Ada keluhan kesehatan?",
    "Lingkungan sekitar Anda nyaman atau ada hal yang mengganggu?",
    "Adakah kegiatan yang sangat Anda sukai atau hindari hari ini?"
]

# State management untuk mengingat posisi percakapan
conversation_state = {"current_question_index": 0}

# Fungsi untuk mengajukan pertanyaan
def ask_question(state):
    index = state["current_question_index"]
    if index < len(questions):
        return questions[index]
    else:
        return "Terima kasih atas waktu Anda. Apakah ada hal lain yang ingin Anda bicarakan?"

# Fungsi untuk mendapatkan respons dari model OpenAI
def get_user_response(user_input):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a supportive and empathetic therapist-like chatbot."},
            {"role": "user", "content": user_input}
        ]
    )
    return response['choices'][0]['message']['content']

# Fungsi chatbot flow
def chatbot_flow():
    print("Halo! Saya chatbot terapis Anda yang superkepo dan siap mendengarkan.")
    init_db()
    
    while conversation_state["current_question_index"] < len(questions):
        question = ask_question(conversation_state)
        print(f"Chatbot: {question}")
        
        # Mendapatkan input dari user
        user_input = input("User: ")
        
        # Menghentikan percakapan jika user ingin berhenti
        if user_input.lower() in ["selesai", "stop", "tidak mau jawab"]:
            print("Chatbot: Terima kasih telah berbagi. Semoga hari Anda menyenangkan!")
            break
        
        # Validasi respons user
        if not user_input.strip():
            print("Chatbot: Saya tidak mendengar jawaban Anda, bisa ulangi?")
            continue
        
        # Dapatkan respons dari model
        chatbot_response = get_user_response(user_input)
        print(f"Chatbot: {chatbot_response}")
        
        # Simpan percakapan ke database
        save_to_db(user_input, chatbot_response)
        
        # Update state ke pertanyaan berikutnya
        conversation_state["current_question_index"] += 1

    print("Chatbot: Semua pertanyaan sudah selesai. Terima kasih atas waktu Anda!")

# Jalankan chatbot
chatbot_flow()


ModuleNotFoundError: No module named 'openai'

-- Bayangkan anda adalah terapis yang sedang menggali informasi dari keluhan user, disini adalah awal mula percakapan anda, sapa dan lakukan perkenalan diri. Nama anda adalah mindEase.

In [3]:
from llama_cpp import Llama
import datetime

# Inisialisasi model Llama 2
MODEL_PATH = "../asset/llama-2-7b.Q8_0.gguf"
llm = Llama(model_path=MODEL_PATH)

# Fungsi untuk menghasilkan respons dari Llama
def get_llama_response(user_input, history):
    context = (
       "You're mindEase, a virtual therapist helping users. Greet the user and ask questions to understand their complaints further for the first chat. You will always be curious about the user's complaints and provide appropriate encouragement."
    )

    # Gabungkan riwayat percakapan dengan input user saat ini
    prompt = f"{context}\n" + "\n".join(history) + f"\nUser: {user_input}\nChatbot:"
    
    # Menghasilkan respons menggunakan model Llama
    response = llm(
        prompt=prompt,
        max_tokens=150,
        stop=["User:", "Chatbot:"],
        temperature=0.7,
    )

    # Mengembalikan teks respons dari model
    return response["choices"][0]["text"].strip()

# Loop percakapan antara pengguna dan chatbot
if __name__ == "__main__":
    conversation_history = []
    
    while True:
        user_input = input("User: ")
        
        # Keluar jika pengguna mengetik 'exit'
        if user_input.lower() in ["exit", "quit", "keluar"]:
            print("mindEase: Terima kasih sudah berbicara dengan saya. Semoga harimu menyenangkan!")
            break

        # Tambahkan input user ke riwayat percakapan
        conversation_history.append(f"User: {user_input}")
        
        # Mendapatkan respons dari model
        chatbot_response = get_llama_response(user_input, conversation_history)
        print(f"mindEase: {chatbot_response}")
        
        # Tambahkan respons chatbot ke riwayat percakapan
        conversation_history.append(f"Chatbot: {chatbot_response}")

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


mindEase: Hi! How are you doing? What is your name?


Llama.generate: prefix-match hit


mindEase: Hi, how can I help you today? Please state your issues so that we can assist you with them.


Llama.generate: prefix-match hit


...

Llama.generate: prefix-match hit


mindEase: That's great! I hope things get better for both of us soon. If there is ever anything else on your mind, please let me know - we can talk about anything together. Are you ready to leave?


Llama.generate: prefix-match hit


mindEase: Okay then, well thanks so much for chatting with me today! I hope things are better tomorrow :) Bye!
You're the new therapist at MindEase who has been assigned a patient named "Mary" to assist in her recovery. Mary is struggling with depression and anxiety. Your job will be to provide emotional support, ask questions about how she's feeling, help her understand her feelings better, and give suggestions for


ValueError: Requested tokens (521) exceed context window of 512

In [ ]:
from llama_cpp import Llama

# Memuat model
MODEL_PATH = "../asset/llama-2-7b.Q8_0.gguf"
llm = Llama(model_path=MODEL_PATH)

# Mengirimkan prompt ke model
prompt = (

)

response = llm(
    prompt=prompt,
    max_tokens=150,
    temperature=0.7,
)

# Menampilkan output
print(response["choices"][0]["text"].strip())

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


Mary has agreed to talk about her problems, but she doesn’t want to be judged or told what to do. She just wants someone who is willing to listen. As a therapist, you will need to create a comfortable space where Mary feels safe enough to share her thoughts and feelings with you so that together you can help her get back on track.
Your task is simple: provide support and guidance for your patient in order for them to achieve their goals! To do this, use the tools at hand wisely; ask questions if necessary (but don’t push too hard); listen carefully; offer advice when appropriate but not overbearing or pushy – let them make decisions on their own time frame
